### This works for the naive approach, where all yearly subscriptions start at 1.1. and end at 31.12.

In [23]:
import pulp

### Set up Mickey-Mouse Data

In [31]:
# Data setup
# Sets
packages = ["P1", "P2"]  # Example package IDs
months = ["Jan23", "Feb23", "Mar23", "Apr23",
          'Jan24', 'Mar24']  # Months in which games occur -> precomputed
years = ["2023", "2024"]  # Years in which games occur -> precomputed

games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']  # Games of interest (Main Input)


# Cost Parameters
C_month = {'P1': 30, 'P2': 20}  # Monthly prices 
C_year = {'P1': 300, 'P2': 180}  # Yearly prices


# Example game coverage
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'],'G5': ['P1', 'P2'],
       'G6': ['P1'], 'G7': ['P1', 'P2']} # Packages that cover each game -> Needs to be pre-computed!!

m_g = {"G1": "Jan23", "G2": "Feb23", "G3": "Mar23", "G4": "Apr23", "G5": "Apr23", 'G6': 'Jan24', 'G7': 'Mar24'} # month of the games
y_g = {"G1": "2023", "G2": "2023", "G3": "2023", "G4": "2023", "G5": "2023", 'G6': '2024', 'G7': '2024'}  # Year of each game

# Packages that can cover games in a month/year
P_m = {m: [p for p in packages if any(p in P_g[g] and m_g[g] == m for g in games)] for m in months}
P_y = {y: [p for p in packages if any(p in P_g[g] and y_g[g] == y for g in games)] for y in years}

P_m

{'Jan23': ['P1'],
 'Feb23': ['P1'],
 'Mar23': ['P1', 'P2'],
 'Apr23': ['P1', 'P2'],
 'Jan24': ['P1'],
 'Mar24': ['P1', 'P2']}

### Define Model

In [32]:
# Model
model = pulp.LpProblem("Streaming_Package_Optimization", pulp.LpMinimize)

# Decision variables
x_month = {p: pulp.LpVariable(f"x_month_{p}", lowBound=0, cat='Integer') for p in packages}
x_year = {p: pulp.LpVariable(f"x_year_{p}", lowBound=0, cat='Integer') for p in packages}
z_month = {(p, m): pulp.LpVariable(f"z_month_{p}_{m}", cat='Binary') for p in packages for m in months}
z_year = {(p, y): pulp.LpVariable(f"z_year_{p}_{y}", cat='Binary') for p in packages for y in years}

# Objective function: Minimize total cost
model += pulp.lpSum(C_month[p] * x_month[p] + C_year[p] * x_year[p] for p in packages)

# Constraints
# 1. Game coverage
for g in games:
    model += pulp.lpSum(z_month[p, m_g[g]] for p in P_g[g]) + pulp.lpSum(z_year[p, y_g[g]] for p in P_g[g]) >= 1

# 2. Monthly subscription activation
for p in packages:
    for m in months:
        model += z_month[p, m] <= x_month[p]

# 3. Yearly subscription activation
for p in packages:
    for y in years:
        model += z_year[p, y] <= x_year[p]

# 4. Monthly activations limited to subscriptions
for p in packages:
    model += pulp.lpSum(z_month[p, m] for m in months) <= x_month[p]

# 5. Yearly activations limited to subscriptions
for p in packages:
    model += pulp.lpSum(z_year[p, y] for y in years) <= x_year[p]


In [33]:
# Solve the model
status = model.solve(pulp.PULP_CBC_CMD())

# Output the results
print("Status:", pulp.LpStatus[status])
print("Total Cost:", pulp.value(model.objective))

for p in packages:
    print(f"Package {p}: Monthly subscriptions = {x_month[p].varValue}, Yearly subscriptions = {x_year[p].varValue}")

for p in packages:
    for m in months:
        if z_month[p, m].varValue > 0:
            print(f"Package {p} active in month {m}")

for p in packages:
    for y in years:
        if z_year[p, y].varValue > 0:
            print(f"Package {p} active in year {y}")

Status: Optimal
Total Cost: 150.0
Package P1: Monthly subscriptions = 3.0, Yearly subscriptions = 0.0
Package P2: Monthly subscriptions = 3.0, Yearly subscriptions = 0.0
Package P1 active in month Jan23
Package P1 active in month Feb23
Package P1 active in month Jan24
Package P2 active in month Mar23
Package P2 active in month Apr23
Package P2 active in month Mar24
